In [110]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from model import BiGRU, CustomLoss, preprocess_data, CSVDataset, plot_heatmaps, preprocess_for_test, get_normalization_values, normalize_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
# Point the save_folder to your predictions
save_folder = '/home/jonathan/Desktop/JARVIS-HybridNet/projects/May11/predictions/predictions3D/Predictions_3D_20230512-160305'
csv_file = save_folder + '/data3D.csv'
data = pd.read_csv(csv_file)

/tmp/ipykernel_988189/3870571526.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_file)


In [ ]:
norm_values = get_normalization_values(data)
processed_data = preprocess_data(data, norm_values=norm_values)

# Set the model parameters
input_size = processed_data.shape[2]
hidden_size = 1024
num_layers = 1
dropout = 0.2
output_size = processed_data.shape[2]

# Initialize the model and loss function
model = BiGRU(input_size, hidden_size, num_layers, output_size, dropout=dropout)
criterion = CustomLoss()

# Set the learning rate
learning_rate = 0.001

# Initialize the Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Set the number of epochs
epochs = 500
batch_size = 32

if torch.cuda.is_available():
    device = torch.device("cuda")
    model.cuda()
else:
    device = torch.device("cpu")

model.train()

# Train the model
for epoch in range(epochs):
    processed_data = preprocess_data(data, norm_values=norm_values, batch_size=batch_size)

    # Instantiate the custom dataset
    dataset = CSVDataset(processed_data)

    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for batch_idx, (inputs, targets, confidence) in enumerate(dataloader):

        # Move tensors to the appropriate device (if using GPU)
        inputs, targets, confidence = inputs.to(device), targets.to(device), confidence.to(device)

        # Reset the gradients to zero
        optimizer.zero_grad()

        # Model output
        output = model(inputs)

        # Calculate loss
        loss = criterion(output, targets, confidence)

        # Backpropagation
        loss.backward()

        # Update the weights
        optimizer.step()

        # Print the loss for the current batch
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch: 1/500, Loss: 0.5251713991165161
Epoch: 2/500, Loss: 0.1712782233953476
Epoch: 3/500, Loss: 0.04741213470697403
Epoch: 4/500, Loss: 0.09679674357175827
Epoch: 5/500, Loss: 0.05157368257641792
Epoch: 6/500, Loss: 0.03590139001607895
Epoch: 7/500, Loss: 0.03842749446630478
Epoch: 8/500, Loss: 0.037597160786390305
Epoch: 9/500, Loss: 0.037142761051654816
Epoch: 10/500, Loss: 0.03266436234116554
Epoch: 11/500, Loss: 0.024364100769162178
Epoch: 12/500, Loss: 0.016751114279031754
Epoch: 13/500, Loss: 0.021320553496479988
Epoch: 14/500, Loss: 0.022710613906383514
Epoch: 15/500, Loss: 0.019821632653474808
Epoch: 16/500, Loss: 0.01280167419463396
Epoch: 17/500, Loss: 0.01293257437646389
Epoch: 18/500, Loss: 0.011456234380602837
Epoch: 19/500, Loss: 0.014420527964830399
Epoch: 20/500, Loss: 0.014393510296940804
Epoch: 21/500, Loss: 0.014930264092981815
Epoch: 22/500, Loss: 0.011181199923157692
Epoch: 23/500, Loss: 0.010414975695312023
Epoch: 24/500, Loss: 0.0076769208535552025
Epoch: 25/50

In [ ]:
processed_data = preprocess_for_test(data, norm_values=norm_values)
# Instantiate the custom dataset
dataset = CSVDataset(processed_data)
# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=1)
for batch_idx, (inputs, targets, confidence) in enumerate(dataloader):
    inputs, targets, confidence = inputs.to(device), targets.to(device), confidence.to(device)
    model.eval()
    with torch.no_grad():
        output = model(inputs).cpu()
    plot_heatmaps(inputs[0,:,0:8].cpu(), output[0,:,0:8].detach().numpy())

In [ ]:
data_aug = pd.DataFrame.copy(data)
# un-normalize
data_aug.loc[1:,:] = normalize_data(output.detach().numpy(), norm_values, forward=False)

# if you don't want to interpolate time points with no centerdetect
if False:
    nan_indices = data.isna().stack()[lambda x: x].index.tolist()
    for row, col in nan_indices:
        data_aug.at[row, col] = pd.NA

data_aug.to_csv(save_folder + '/data3D_smooth.csv', index=False)

In [ ]:
plot_heatmaps(model.gru.all_weights[0][0].cpu().detach().numpy(), model.gru.all_weights[0][1].cpu().detach().numpy())